In [1]:
import pandas as pd
import numpy as np
import json
from ast import literal_eval
import matplotlib.pyplot as plt
from datetime import datetime
import collections, functools, operator
import re

### The cell below load the Safegraph data and parse them

- Now itliteral_eval up for NYC (Bronx, Kings, New York, Queens, Richmond counties)
- The three datasets are for 2020/04, 2019/04 and 2020/10
- The files are exported to *csv.tar.gz in the datasets directory
- DO NOT run the following cell if you do not want to reparse the data
- For futher data explorations just use the *csv.tar.gz in the datasets directory

In [2]:
def parseData(year, month, state, counties, city):
    # For SafeGraph PlaceID to county conversion
    df_CBG = pd.read_csv('../datasets/placeToCBGMay/placeCountyCBG.csv')
    
    df_1 = pd.read_csv('../datasets/Places_Patterns/'+year+'/'+month+'/patterns-part1.csv.gz', compression='gzip')
    df_2 = pd.read_csv('../datasets/Places_Patterns/'+year+'/'+month+'/patterns-part2.csv.gz', compression='gzip')
    df_3 = pd.read_csv('../datasets/Places_Patterns/'+year+'/'+month+'/patterns-part3.csv.gz', compression='gzip')
    df_4 = pd.read_csv('../datasets/Places_Patterns/'+year+'/'+month+'/patterns-part4.csv.gz', compression='gzip')
    df_T = pd.concat([df_1, df_2, df_3, df_4])

    df_TC = pd.merge(df_CBG[df_CBG['state']==state], df_T[df_T['region']==state], on='safegraph_place_id')
    df_TC['countyName'] = df_TC['countyName'].apply(lambda x: x[:-7])

    df_F = df_TC[df_TC['countyName'].isin(counties)]
    df_F.drop(columns=['countyFIPS', 'stateFIPS', 'CBGFIPS', 'region', 'safegraph_brand_ids'], inplace=True)
    df_F.reset_index(drop=True, inplace=True)
    df_F.to_csv('../datasets/'+city+'/'+city+'_'+year+'_'+month+'.csv.tar.gz', index=False, compression='gzip')
    
    return df_F

def parseCore(year, month, state, counties, city):
    # For SafeGraph PlaceID to county conversion
    df_CBG = pd.read_csv('../datasets/placeToCBGMay/placeCountyCBG.csv')
    
    df_1 = pd.read_csv('../datasets/Core/'+year+'/'+month+'/core_poi-part1.csv.gz', compression='gzip')
    df_2 = pd.read_csv('../datasets/Core/'+year+'/'+month+'/core_poi-part2.csv.gz', compression='gzip')
    df_3 = pd.read_csv('../datasets/Core/'+year+'/'+month+'/core_poi-part3.csv.gz', compression='gzip')
    df_4 = pd.read_csv('../datasets/Core/'+year+'/'+month+'/core_poi-part4.csv.gz', compression='gzip')
    df_5 = pd.read_csv('../datasets/Core/'+year+'/'+month+'/core_poi-part5.csv.gz', compression='gzip')
    df_T = pd.concat([df_1, df_2, df_3, df_4, df_5])

    df_TC = pd.merge(df_CBG[df_CBG['state']==state], df_T[df_T['region']==state], on='safegraph_place_id')
    df_TC['countyName'] = df_TC['countyName'].apply(lambda x: x[:-7])

    df_F = df_TC[df_TC['countyName'].isin(counties)]
    df_F.drop(columns=['countyFIPS', 'stateFIPS', 'CBGFIPS', 'phone_number', 'iso_country_code', 'region', 'safegraph_brand_ids'], inplace=True)
    df_F.reset_index(drop=True, inplace=True)
    df_F.to_csv('../datasets/'+city+'/Core-'+city+'_'+year+'_'+month+'.csv.tar.gz', index=False, compression='gzip')
    return df_F

 
def getCity(state, counties, city):
    df_dict = {}
    for year in ['2019', '2020']:
        for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
            df_dict[year+'_'+month] = parseData(year, month, state, counties, city)
    
    for year in ['2021']:
        for month in ['01', '02', '03', '04']:
            df_dict[year+'_'+month] = parseData(year, month, state, counties, city)

    dfC = parseCore('2020', '04', state, counties, city)
    return df_dict, dfC

## New York

In [56]:
parsedata = True
if parsedata:
    getCity('NY', ['Bronx', 'Kings', 'New York', 'Queens', 'Richmond'], 'NYC')

## Seattle

In [57]:
parsedata = True
if parsedata:
    df_dict_Seattle, dfC_Seattle = getCity('WA', ['King', 'Snohomish', 'Pierce'], 'SeattleMetro')

## Boston

In [58]:
parsedata = True
if parsedata:
    df_dict_Boston, dfC_Boston = getCity('MA', ['Suffolk', 'Middlesex'], 'Boston')

## Los Angeles

In [4]:
parsedata = True
if parsedata:
    df_dict_LA, dfC_LA = getCity('CA', ['Los Angeles'], 'LosAngeles')

## Chicago

In [5]:
parsedata = True
if parsedata:
    df_dict_Chicago, dfC_Chicago = getCity('IL', ['Cook'], 'Chicago')

## Houston

In [6]:
parsedata = True
if parsedata:
    df_dict_Houston, dfC_Houston = getCity('TX', ['Harris'], 'Houston')

## Phoenix

In [7]:
parsedata = True
if parsedata:
    df_dict_Phoe, dfC_Phoenix = getCity('AZ', ['Maricopa'], 'Phoenix')